In [13]:
# Connectem al drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Afegim el path del directori
%cd /content/drive/MyDrive/Colab_Notebooks/PRA VD/

/content/drive/MyDrive/Colab_Notebooks/PRA VD


In [15]:
# Importem les llibreries 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import functools as ft

In [16]:
# Definim una funció per obtenir els diputats de cada legislatura
def diputats_legislatura(legislatura, diputat):
  url = "https://www.congreso.es/busqueda-de-diputados?p_p_id=diputadomodule&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_diputadomodule_mostrarFicha=true&codParlamentario="+str(diputat)+"&idLegislatura="+legislatura+"&mostrarAgenda=false"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  noms = soup.find_all(class_="nombre-dip")
  diputats = []
  for nom in noms:
    diputats.append(nom.text.split("\t")[4].split("\r")[0])
  return diputats[0]

In [ ]:
# LA DESCÀRREGA DE LES DADES TRIGA HORES

# Obtenim els 350 diputats inicials de cada legislatura
legislatures = []
for i,j in enumerate(["C", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX", "X", "XI", "XII", "XIII", "XIV"]):
  print("Legislatura: ", j)
  legislatures.append(pd.DataFrame({"Diputats": [diputats_legislatura(j, n) for n in range(1,351)], "Leg": i}))

# Guardem les dades de cada legislatura
for n in range(15):
  legislatures[n].to_csv("leg_" + str(n) + ".csv", index = False)

Legislatura:  C
Legislatura:  I
Legislatura:  II
Legislatura:  III
Legislatura:  IV
Legislatura:  V
Legislatura:  VI
Legislatura:  VII
Legislatura:  VIII
Legislatura:  IX
Legislatura:  X
Legislatura:  XI
Legislatura:  XII
Legislatura:  XIII
Legislatura:  XIV


In [17]:
# Recuperem els arxius guardats
leg_C = pd.read_csv("leg_0.csv")
leg_C.rename(columns={"Leg": "Leg_C"}, inplace=True)
leg_I = pd.read_csv("leg_1.csv")
leg_I.rename(columns={"Leg": "Leg_I"}, inplace=True)
leg_II = pd.read_csv("leg_2.csv")
leg_II.rename(columns={"Leg": "Leg_II"}, inplace=True)
leg_III = pd.read_csv("leg_3.csv")
leg_III.rename(columns={"Leg": "Leg_III"}, inplace=True)
leg_IV = pd.read_csv("leg_4.csv")
leg_IV.rename(columns={"Leg": "Leg_IV"}, inplace=True)
leg_V = pd.read_csv("leg_5.csv")
leg_V.rename(columns={"Leg": "Leg_V"}, inplace=True)
leg_VI = pd.read_csv("leg_6.csv")
leg_VI.rename(columns={"Leg": "Leg_VI"}, inplace=True)
leg_VII = pd.read_csv("leg_7.csv")
leg_VII.rename(columns={"Leg": "Leg_VII"}, inplace=True)
leg_VIII = pd.read_csv("leg_8.csv")
leg_VIII.rename(columns={"Leg": "Leg_VIII"}, inplace=True)
leg_IX = pd.read_csv("leg_9.csv")
leg_IX.rename(columns={"Leg": "Leg_IX"}, inplace=True)
leg_X = pd.read_csv("leg_10.csv")
leg_X.rename(columns={"Leg": "Leg_X"}, inplace=True)
leg_XI = pd.read_csv("leg_11.csv")
leg_XI.rename(columns={"Leg": "Leg_XI"}, inplace=True)
leg_XII = pd.read_csv("leg_12.csv")
leg_XII.rename(columns={"Leg": "Leg_XII"}, inplace=True)
leg_XIII = pd.read_csv("leg_13.csv")
leg_XIII.rename(columns={"Leg": "Leg_XIII"}, inplace=True)
leg_XIV = pd.read_csv("leg_14.csv")
leg_XIV.rename(columns={"Leg": "Leg_XIV"}, inplace=True)

In [18]:
# Agreguem la informació en un únic dataframe
dfs = [leg_C, leg_I, leg_II, leg_III, leg_IV, leg_V, leg_VI, leg_VII, leg_VIII, leg_IX, leg_X, leg_XI, leg_XII, leg_XIII, leg_XIV]
diputats = ft.reduce(lambda left, right: pd.merge(left, right, how="outer", on="Diputats"), dfs)

# Adequem les dades
diputats.loc[diputats["Leg_C"]==0, "Leg_C"] = 1
diputats = diputats.fillna(0)
for n, leg in enumerate(["Leg_II", "Leg_III", "Leg_IV", "Leg_V", "Leg_VI", "Leg_VII", "Leg_VIII", "Leg_IX", "Leg_X", "Leg_XI", "Leg_XII", "Leg_XIII", "Leg_XIV"]):
  diputats.loc[diputats[leg]==n+2, leg] = 1
legislatures = ["Leg_C", "Leg_I", "Leg_II", "Leg_III", "Leg_IV", "Leg_V", "Leg_VI", "Leg_VII", "Leg_VIII", "Leg_IX", "Leg_X", "Leg_XI", "Leg_XII", "Leg_XIII", "Leg_XIV"]
diputats[legislatures] = diputats[legislatures].astype(int)

In [19]:
# Obtenim noms de pila 
nom_pila = []
for nom in diputats["Diputats"]:
  nom_pila.append(nom.split(",")[1].split(" ")[1])

# Afegim manualmente llista amb noms femenins 
noms_femenins = ["Adoración", "Alexandra", "Amalia", "Amalur", "Amaya", "Angustias", 
                "Anna", "Arantza", "Ariagona", "Ascensión", "Assumpció", "Belarmina",
                "Carmina", "Catalina", "Celia", "Claudina", "Clemencia", "Clementina",
                "Clàudia", "Coloma", "Concepció", "Coralí", "Dolors", "Donelia", "Dorinda",
                "Edelmira", "Elisabeth", "Elisenda", "Esperanza", "Esperança", "Ester",
                "Estrella", "Eulàlia", "Florentina", "Francisca", "Gemma", "Georgina", 
                "Idoia", "Ildefonsa", "Ione", "Iratxe", "Isaura", "Itziar", "Izaskun",
                "Joaquima", "Josefina", "Josune", "Juliana", "Koro", "Lentxu", "Lidia",
                "Loreto", "Loyola", "Lucila", "Lucita", "Ludivina", "Luisa", "Lídia",
                "Maravillas", "Margalida", "Maria", "Marian", "Mariana", "Martirio",
                "Melisa", "Melissa", "Mercè", "Mertxe", "Mirella", "Miren", "Montserrat",
                "Míriam", "Mònica", "Nayua", "Nazaria", "Nona", "Obdulia", "Onintza",
                "Orlena", "Petronila", "Presentación", "Riansares", "Rosina", "Rut",
                "Saturnina", "Severa", "Soledad", "Sonia", "Sonsoles", "Susanna", "Sònia",
                "Tristana", "Vicenta", "Violante", "Águeda", "Èlia", "Adela", "Adriana", 
                "Aina", "Alicia", "Amparo", "Ana", "Antonia", "Ariagona", "Aurora", "Beatriz",
                "Begoña", "Belén", "Berta", "Blanca", "Carla", "Carlota", "Carme", "Carmela",
                "Carmen", "Carolina", "Cayetana", "Celinda", "Concepción", "Cristina", "Cándida",
                "Dolores", "Edurne", "Elena", "Eloísa", "Elvira", "Encarnación", "Esther",
                "Eulalia", "Eva", "Francesca", "Fuensanta", "Gema", "Gloria", "Helena",
                "Inmaculada", "Inés", "Irene", "Irma", "Isabel", "Josefa", "Juana", "Julia",
                "Laia", "Laura", "Leire", "Lourdes", "Lucía", "Macarena", "Magdalena", "Maite",
                "Manuela", "Mar", "Margarita", "Maribel", "Marina", "Mariona", "Marisa", "Marisol",
                "Marta", "Martina", "María", "Matilde", "Mercé", "Meritxell", "Micaela", "Milagros",
                "Mireia", "Miriam", "Montse", "Mª", "Mónica", "Noelia", "Noemí", "Norma", "Ofelia", 
                "Olga", "Olivia", "Onintza","Palmira", "Paloma", "Patricia", "Pilar", "Rafaela", 
                "Raquel", "Remedios", "Reyes", "Rita", "Rocío", "Rosa", "Rosalía", "Rosana",
                "Rosario", "Roser", "Sandra", "Sara", "Silvia", "Sofía", "Sol", "Soledad", "Soraya",
                "Tania", "Susana", "Teresa", "Teófila", "Trinidad", "Tristana", "Victoria", 
                "Virtudes", "Yolanda", "Zaida", "Àngela", "Ángela", "Ángeles"]

# Classifiquem gènere
genere = []
for nom in nom_pila:
  if nom in noms_femenins:
    genere.append("Femení")
  else:
    genere.append("Masculí")

# Afegim la columna "Gènere" 
diputats["Gènere"] = genere

In [20]:
# Guardem dataset
diputats.to_csv("Diputats.csv", index = False)

In [21]:
# Dades diputades
femeni = diputats[diputats["Gènere"]=="Femení"]

In [22]:
# Evolució del número de diputades
num = []
for leg in legislatures:
  num.append(np.sum(femeni[leg]))

evolucio = pd.DataFrame({"Legislatura": legislatures, "Número": num})

evolucio.to_csv("Evolució.csv", index = False)

In [23]:
# Presències de diputades al Congrés 
num_leg = []
for n in range(len(femeni)):
  num_leg.append(np.sum(femeni.iloc[n,1:16]))

femeni["Presències"] = num_leg
femeni = femeni.sort_values("Presències", ascending=False)

presencies = pd.DataFrame({"Diputades": femeni["Diputats"], "Legislatures": femeni["Presències"], "Constituent": femeni["Leg_C"], "I": femeni["Leg_I"],
                           "II": femeni["Leg_II"], "III": femeni["Leg_III"], "IV": femeni["Leg_IV"], "V": femeni["Leg_V"], "VI": femeni["Leg_VI"],
                           "VII": femeni["Leg_VII"], "VIII": femeni["Leg_VIII"], "IX": femeni["Leg_IX"], "X": femeni["Leg_X"], "XI": femeni["Leg_XI"],
                           "XII": femeni["Leg_XII"], "XIII": femeni["Leg_XIII"], "XIV": femeni["Leg_XIV"]})

presencies= presencies.replace([0], " ")
presencies= presencies.replace([1], "X")
presencies["Legislatures"] = presencies["Legislatures"].replace(["X"], 1)

presencies.to_csv("Presències.csv", index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
